In [14]:
import pandas as pd
import numpy as np
import math
import haversine as hs
import requests
import json
from haversine import Unit
import getFlightData

In [2]:
try:  # import tracks from csv
    df_path = pd.read_csv('path.csv')
    df_airSpeed = pd.read_csv('air_speed.csv')
except OSError:
    print("Could not connect to database...")

In [3]:
def clc_dist_between_2points(lat1, lon1, lat2, lon2):
    coordinate1 = (lat1, lon1)
    coordinate2 = (lat2, lon2)
    dist_metre = hs.haversine(coordinate1, coordinate2, unit=Unit.METERS)
    return dist_metre

In [4]:
def clc_azimuth(lat1, lon1, lat2, lon2):
    dL = lon2 - lon1
    X = math.cos(lat2) * math.sin(dL)
    Y = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(dL)
    bearing_rad = np.arctan2(X, Y)  # radian
    bearing_deg = ((np.degrees(bearing_rad) + 360) % 360)  # degrees
    return bearing_deg

In [5]:
def get_Aspeed(weight):
    weight_modol = weight - (weight % 50)  # rounding down
    try:
        Aspeed = int(df_airSpeed['air speed'][df_airSpeed[df_airSpeed['weight'] == weight_modol].index])  # air speed in csv
    except:
        print("fuel is to low- good luck coming back")
        Aspeed = 0
    return Aspeed

In [6]:
def get_weather(lat, lon, request):
    api_key = "c80b790a2fd8650bc690c41a4c001f0a"
    url = "https://api.openweathermap.org/data/2.5/onecall?lat=%s&lon=%s&appid=%s&units=metric" % (lat, lon, api_key)
    response = requests.get(url)
    data = json.loads(response.text)
    weather_request = data["current"][request]
    return weather_request

In [7]:
def clc_Gspeed(az, wind_speed, wind_deg, Aspeed):
    Gspeed = Aspeed + wind_speed * math.cos((wind_deg + 180) - az)  #pleas confirm me
    return Gspeed
# add yaw

In [8]:
def clc_leg_FlightTime_FuelWeist(leg_number, wind_ref_point, aircraftType, current_fuel):
    aircraft_weight = 10  # (need to add aircraft weight by aircraft type)
    weight_tot = current_fuel + aircraft_weight  # (need to add aircraft weight by aircraft type)

    distance = clc_dist_between_2points(df_path['lat_a'][leg_number], df_path['lon_a'][leg_number], df_path['lat_b'][leg_number], df_path['lon_b'][leg_number])
    high_diff = df_path['high_b(m)'][leg_number] - df_path['high_a(m)'][leg_number]
    az = clc_azimuth(df_path['lat_a'][leg_number], df_path['lon_a'][leg_number], df_path['lat_b'][leg_number], df_path['lon_b'][leg_number])
    startTemp, endTemp = get_weather(df_path['lat_a'][leg_number], df_path['lon_a'][leg_number], "temp"), get_weather(df_path['lat_b'][leg_number], df_path['lon_b'][leg_number], "temp")

    if high_diff < 0:
        flightPattern = "descend"
    if high_diff > 0:
        flightPattern = "climb"
    # avgROC, avgFC = getFlightData.getFlightLegData(aircraftType, flightPattern, weight_tot, current_fuel, startTemp, endTemp, df_path['high_a(m)'][leg_number], df_path['high_b(m)'][leg_number])
    # avgROC_lev, avgFC_lev = getFlightData.getFlightLegData(aircraftType, ["leveled flight"], weight_tot, current_fuel, startTemp, endTemp, df_path['high_a(m)'][leg_number], df_path['high_b(m)'][leg_number])
    avgROC = 10  ##
    avgFC = 0.007  ##
    avgFC_lev = 0.006  ##

    if (wind_ref_point == 'start'):  # option for later
        wind_speed = get_weather(df_path['lat_a'][leg_number], df_path['lon_a'][leg_number], "wind_speed")
        wind_deg = get_weather(df_path['lat_a'][leg_number], df_path['lon_a'][leg_number], "wind_deg")
    else:
        wind_speed = get_weather(df_path['lat_b'][leg_number], df_path['lon_b'][leg_number], "wind_speed")
        wind_deg = get_weather(df_path['lat_b'][leg_number], df_path['lon_b'][leg_number], "wind_deg")
    Aspeed = get_Aspeed(aircraft_weight + current_fuel)
    Gspeed = clc_Gspeed(az, wind_speed, wind_deg, Aspeed)

    # calculate flight time from a to b
    time_dist = (distance / Gspeed) * 100 / 6  # 6min/100 -- time of flight from point a to b in minutes
    time_alt = high_diff / avgROC  # min -- time to climb or descend from point 1 to point b in minutes
    time_of_leg = max(time_dist, time_alt)

    # Fuel consumption from a to b
    if time_of_leg == time_alt:
        fuel_decrease = avgFC * time_of_leg
    elif time_of_leg == time_dist:
        fuel_decrease = avgFC * time_alt
        fuel_decrease += avgFC_lev * (time_dist - time_alt)
    new_fuel = current_fuel - fuel_decrease
    if new_fuel <= 0:
        print("out of fuel")
        new_fuel = 0

    # add data to dataFrame
    df_path.loc[leg_number, 'dist'] = distance
    df_path.loc[leg_number, 'high_diff'] = high_diff
    df_path.loc[leg_number, 'az'] = az
    df_path.loc[leg_number, 'Gspeed'] = Gspeed
    df_path.loc[leg_number, 'Aspeed'] = Aspeed
    df_path.loc[leg_number, 'time_dist'] = time_dist
    df_path.loc[leg_number, 'time_climb/descend'] = time_alt
    df_path.loc[leg_number, 'time_of_leg'] = time_of_leg
    df_path.loc[leg_number, 'current_fuel'] = new_fuel

    return time_of_leg, new_fuel


In [9]:
def clc_path_FlightTime_FuelWeist(start_fuel, aircraftType):
    leg_number = int(len(df_path['lat_a']))
    wind_ref_point = 'start'  ###
    current_fuel = start_fuel
    total = 0
    for i in range(leg_number):
        time_of_leg, current_fuel = clc_leg_FlightTime_FuelWeist(i, wind_ref_point, aircraftType, current_fuel)
        total += time_of_leg
    # add total time to csv at the last row of time_of_leg
    df_path.loc[leg_number+1, 'time_of_leg'] = total
    df_path.to_csv("path.csv")  #update csv
    return total, current_fuel

In [15]:
test_total, test_fuel = clc_path_FlightTime_FuelWeist(550, 'Airbus')
print(test_total, test_fuel)